In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


In [3]:
df =pd.read_csv("energydata_complete.csv")

In [4]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [6]:
df.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Appliances,19735.0,97.694958,102.524891,10.000000,50.000000,60.000000,100.000000,1080.000000
lights,19735.0,3.801875,7.935988,0.000000,0.000000,0.000000,0.000000,70.000000
T1,19735.0,21.686571,1.606066,16.790000,20.760000,21.600000,22.600000,26.260000
RH_1,19735.0,40.259739,3.979299,27.023333,37.333333,39.656667,43.066667,63.360000
T2,19735.0,20.341219,2.192974,16.100000,18.790000,20.000000,21.500000,29.856667
RH_2,19735.0,40.420420,4.069813,20.463333,37.900000,40.500000,43.260000,56.026667
T3,19735.0,22.267611,2.006111,17.200000,20.790000,22.100000,23.290000,29.236000
RH_3,19735.0,39.242500,3.254576,28.766667,36.900000,38.530000,41.760000,50.163333
T4,19735.0,20.855335,2.042884,15.100000,19.530000,20.666667,22.100000,26.200000
RH_4,19735.0,39.026904,4.341321,27.660000,35.530000,38.400000,42.156667,51.090000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [10]:
df.shape

(19735, 29)

In [12]:
X = df[['T2']]  # Temperature in the living room (Celsius)
y = df['T6']    # Temperature outside the building


In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Fit a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [15]:
# Predict the target variable on the test set (question 17)
y_pred = model.predict(X_test)

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the RMSE rounded to 3 decimal places
print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")

Root Mean Squared Error (RMSE): 3.633


In [36]:
# copy the dataset to preserve the original dataset
df_ = df.copy()

In [32]:
df_.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [34]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Remove columns "date" and "lights"
df_c= df_.drop(["date", "lights"], axis=1)

# Define features (X) and target variable (y)
X = df_c.drop("Appliances", axis=1)
y = df_c["Appliances"]

# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Fit a multiple linear regression model using the training set
model = LinearRegression()
model.fit(X_train_normalized, y_train)

# Predict the target variable on the training set
y_pred_train = model.predict(X_train_normalized)

# Calculate Mean Absolute Error (MAE) for training set (Question 18)
mae_train = mean_absolute_error(y_train, y_pred_train)

# Predict the target variable on the test set
y_pred_test = model.predict(X_test_normalized)

# Calculate Mean Absolute Error (MAE) for test set (Question 20)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Calculate Root Mean Squared Error (RMSE) for training set (Question 19)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)

# Calculate Root Mean Squared Error (RMSE) for test set (Question 21)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

# Print the results rounded to three decimal places
print(f"Mean Absolute Error (MAE) for training set: {mae_train:.3f}")
print(f"Mean Absolute Error (MAE) for test set: {mae_test:.3f}")
print(f"Root Mean Squared Error (RMSE) for training set: {rmse_train:.3f}")
print(f"Root Mean Squared Error (RMSE) for test set: {rmse_test:.3f}")

Mean Absolute Error (MAE) for training set: 53.742
Mean Absolute Error (MAE) for test set: 53.643
Root Mean Squared Error (RMSE) for training set: 95.216
Root Mean Squared Error (RMSE) for test set: 93.640


In [35]:
# Train a ridge regression model with default parameters (Question 23)
model_ridge = Ridge()
model_ridge.fit(X_train_normalized, y_train)

# Predict the target variable on the test set using ridge regression
y_pred_test_ridge = model_ridge.predict(X_test_normalized)

# Calculate Root Mean Squared Error (RMSE) for test set with ridge regression
rmse_test_ridge = mean_squared_error(y_test, y_pred_test_ridge, squared=False)

# Print the RMSE for test set with ridge regression
print(f"Root Mean Squared Error (RMSE) for test set with ridge regression: {rmse_test_ridge:.3f}")
# Train a lasso regression model with default parameters 
model_lasso = Lasso()
model_lasso.fit(X_train_normalized, y_train)

# Get the coefficients (weights) of the Lasso model
lasso_weights = model_lasso.coef_

# Count the number of features with non-zero weights(Question 24)
num_nonzero_weights = sum(lasso_weights != 0)

# Predict the target variable on the test set using Lasso regression
y_pred_test_lasso = model_lasso.predict(X_test_normalized)

# Calculate Root Mean Squared Error (RMSE) for test set with Lasso regression (Question 25)
rmse_test_lasso = mean_squared_error(y_test, y_pred_test_lasso, squared=False)

# Print the number of features with non-zero weights and RMSE for test set with Lasso regression
print(f"Number of features with non-zero weights in Lasso regression: {num_nonzero_weights}")
print(f"Root Mean Squared Error (RMSE) for test set with Lasso regression: {rmse_test_lasso:.3f}")

Root Mean Squared Error (RMSE) for test set with ridge regression: 93.709
Number of features with non-zero weights in Lasso regression: 4
Root Mean Squared Error (RMSE) for test set with Lasso regression: 99.424
